In [ ]:
import pandas as pd
# import openpyxl

sourceFile="Family expenses.xlsm"
outputJSON="output.json"

# book = openpyxl.load_workbook(sourceFile, data_only=True, read_only=True)

dfExpense = pd.read_excel(
    sourceFile,
    sheet_name="Expenses DB",
    skiprows=9,
    usecols="J,Q:U",
)
dfIncome = pd.read_excel(
    sourceFile,
    sheet_name="Income DB",
    skiprows=9,
    usecols="J,Q:U",
)

dfExpense["Amount"] *= -1
dfIncome.rename(columns={"Type ": "Category"}, inplace=True)
dfIncome["Store"] = "NA"
dfIncome["Exclude in WE"] = "YES"
dfResult = dfExpense.append(dfIncome)
dfResult.sort_values(by="Date", inplace=True)
dfResult.rename(columns={"Exclude in WE": "weeklyexpense"}, inplace=True)
dfResult["created"] = dfResult["Date"].dt.strftime("%Y-%m-%d")
dfResult["updated"] = dfResult["Date"].dt.strftime("%Y-%m-%d")

dfResult.rename(columns={"Amount": "amount"}, inplace=True)
dfResult.rename(columns={"Store": "store"}, inplace=True)
dfResult.rename(columns={"Category": "category"}, inplace=True)
dfResult.rename(columns={"Description": "description"}, inplace=True)

cols = [
    "created",
    "updated",
    "amount",
    "store",
    "category",
    "weeklyexpense",
    "description",
]
dfResult = dfResult[cols]
dfResult.loc[dfResult.weeklyexpense == "NO", 'weeklyexpense'] = 1
dfResult.loc[dfResult.weeklyexpense == "YES", 'weeklyexpense'] = 0

dfResult.head(100)
dfResult.to_json(orient='records',path_or_buf=outputJSON)
dfResult.count()

In [21]:
import pandas as pd
# import openpyxl
df = pd.read_csv('./sept.csv')
df['created'] = pd.to_datetime(df['created']).dt.strftime("%Y-%m-%d")
df['updated'] =  df['created']

cols = [
    "created",
    "updated",
    "amount",
    "store",
    "category",
    "weeklyexpense",
    "description",
]
df = df[cols]
df.head(100)
df.to_json(orient='records',path_or_buf='./sept.json')


In [14]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import urllib
from os import getenv
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
API_KEY = getenv("API_KEY")
API_KEY_NAME = "access_token"
password = getenv("MSSQL_PASSWORD")
server = getenv("MSSQL_SERVER")
database = getenv("MSSQL_DATABASE")
username = getenv("MSSQL_USERNAME")



driver = "{ODBC Driver 17 for SQL Server}"
conn = f"""Driver={driver};Server=tcp:{server},1433;Database={database};
Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"""
params = urllib.parse.quote_plus(conn)
conn_str = "mssql+pyodbc:///?autocommit=true&odbc_connect={}".format(params)
engine = create_engine(conn_str, echo=True)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db = SessionLocal()
Base = declarative_base()

records = db.execute('select top 10 * from records')

cols = [
        "id",
        "created",
        "updated",
        "amount",
        "store",
        "category",
        "weeklyexpense",
        "description",
    ]

df = pd.DataFrame(records.all(),columns=cols)

df.head(2)

2021-10-10 23:25:14,030 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2021-10-10 23:25:14,031 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-10 23:25:14,206 INFO sqlalchemy.engine.Engine SELECT schema_name()
2021-10-10 23:25:14,208 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ()
2021-10-10 23:25:14,849 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2021-10-10 23:25:14,851 INFO sqlalchemy.engine.Engine [generated in 0.00221s] ()
2021-10-10 23:25:15,054 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-10 23:25:15,056 INFO sqlalchemy.engine.Engine select top 10 * from records
2021-10-10 23:25:15,058 INFO sqlalchemy.engine.Engine [generated in 0.00195s] ()


,id,created,updated,amount,store,category,weeklyexpense,description
0,2A1A7BB9-0349-4E37-A84F-0008B8A4118F,2019-03-05,2019-03-05,-7.9,Coles,Daily Expenses,True,toilet stuff
1,3CA443BD-AA84-4A14-9922-000EF110320A,2018-03-30,2018-03-30,-14.5,Others,Daily Expenses,True,Pizza
